In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [3]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

## Define a Keras model

In this example, the classifier is a simple four-layer Sequential model.

In [4]:
# Define the model.
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

Download and prepare the training data.

In [5]:
(train_images, train_labels), _ = keras.datasets.fashion_mnist.load_data()
#train_images = train_images / 255.0

## Train the model and log data

Before training, define the Keras TensorBoard callback, specifying the log directory. By passing this callback to Model.fit(), you ensure that graph data is logged for visualization in TensorBoard.

In [6]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
model.fit(
    train_images,
    train_labels, 
    batch_size=64,
    epochs=5, 
    callbacks=[tensorboard_callback])

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 56us/sample - loss: 3.3964 - accuracy: 0.3003
Epoch 2/5
60000/60000 [==============================] - 3s 45us/sample - loss: 1.6433 - accuracy: 0.3675
Epoch 3/5
60000/60000 [==============================] - 3s 45us/sample - loss: 1.4843 - accuracy: 0.4044
Epoch 4/5
60000/60000 [==============================] - 3s 45us/sample - loss: 1.3771 - accuracy: 0.4384
Epoch 5/5
60000/60000 [==============================] - 3s 46us/sample - loss: 1.3021 - accuracy: 0.4735


## Op-level graph

Start TensorBoard and wait a few seconds for the UI to load. Select the Graphs dashboard by tapping “Graphs” at the top. 

In [7]:
%tensorboard --logdir logs

ERROR: Failed to start `tensorboard`: [Errno 20] Not a directory:
'tensorboard'

In [8]:
import tensorflow as tf
import datetime
%load_ext tensorboard
# The function to be traced.
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
#stamp = datetime.time.strftime()
logdir = 'logs/func/'
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
%tensorboard --logdir logs/func

ERROR: Failed to start `tensorboard`: [Errno 20] Not a directory:
'tensorboard'